# Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_columns = 10
pd.options.display.max_colwidth = -1

In [3]:
import pickle
from collections import Counter

# Data

In [4]:
data = pd.read_csv('../[Data] Sarcasm-Detection/reddit.csv')

In [5]:
data.shape

(1010826, 10)

In [6]:
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well."
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT


In [7]:
X = data[['parent_comment', 'comment']]
y = data['label']

# Preprocessing

### Text Cleaning

In [8]:
import nltk, re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

remove_punc = lambda x : re.sub(r"\W", ' ', x)

remove_extra_spaces = lambda x : re.sub(r"\s+", ' ', x)

lower_case = lambda x : x.lower()

stop_words = set(nltk.corpus.stopwords.words('english'))
remove_stopwords = lambda x: ' '.join(word for word in x.split() if word not in stop_words)

ps = PorterStemmer()
ps_stem = lambda x: ' '.join(ps.stem(word) for word in x.split())

wnl = WordNetLemmatizer()
wnl_lemmatize = lambda x: ' '.join(wnl.lemmatize(word) for word in x.split())

def tag_pos(x):
    tag_list =  nltk.pos_tag(nltk.word_tokenize(x))
    pos = ""
    for t in tag_list:
        pos += t[0] +'(' + t[1] +')' + ' '
    return pos

def cleanText(x, rsw, stm, lem, tgps):
    x = remove_punc(x)
    x = remove_extra_spaces(x)
    x = lower_case(x)
    if rsw:
        x = remove_stopwords(x)
    if stm:
        x = ps_stem(x)
    if lem:
        x = wnl_lemmatize(x)
    if tgps:
        x = tag_pos(x)
    return x

In [9]:
X['parent_comment'] = X['parent_comment'].apply(lambda x: cleanText(x, True, False, True, False))

D:\Users\Ritvik\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
X['comment'] = X['comment'].apply(lambda x: cleanText(str(x), True, False, True, False))

D:\Users\Ritvik\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
X.head()

,parent_comment,comment
0,yeah get argument point prefer lived nc well,nc nh
1,blazer maverick west 5 6 seed even carry good enough record make playoff east last year,know west team play west team east team right
2,favored win,underdog earlier today since gronk announcement afternoon vega line moved patriot 1
3,deadass kill buzz,meme funny none new york nigga one
4,yep confirm saw tool use made boy easports_mut,could use one tool


### Vectorizing

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
Tvectorizer1 = TfidfVectorizer(ngram_range=(1, 3))

In [14]:
Tvectorizer1.fit(X['parent_comment'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [15]:
Xvect1 = Tvectorizer1.transform(X['parent_comment'])

In [16]:
Tvectorizer2 = TfidfVectorizer(ngram_range=(1,3))

In [17]:
Tvectorizer2.fit(X['comment'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [18]:
Xvect2 = Tvectorizer2.transform(X['comment'])

In [19]:
from scipy.sparse import hstack

In [20]:
Xvect = hstack([Xvect1, Xvect2])

In [21]:
Xvect.shape

(1010826, 21792523)

In [ ]:
pickle.dump(Tvectorizer1, open('v2/Tvect1.pickle', 'wb'))

In [ ]:
pickle.dump(Tvectorizer2, open('v2/Tvect2.pickle', 'wb'))

# Machine Learning

In [22]:
from sklearn.model_selection import cross_val_score

### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression()
scores = cross_val_score(model, Xvect, y, cv=10)
print('Accuracy: %0.2f (+/- %0.2f)' %(scores.mean(), scores.std()))

Accuracy: 0.69 (+/- 0.00)


### Multinomial NB

In [25]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
model = MultinomialNB()
scores = cross_val_score(model, Xvect, y, cv=10)
print('Accuracy: %0.2f (+/- %0.2f)' %(scores.mean(), scores.std()))

Accuracy: 0.68 (+/- 0.00)
